In [ ]:
import numpy as np
import pyomo.environ as pe
import pyomo.opt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
from pyomo.util.infeasible import log_infeasible_constraints
import os
import pandas as pd
import dotenv
import dc_model
import simpler_model

# DC Community

In [ ]:
# Set default behaviour
default_behaviour = pe.Constraint.Skip

In [ ]:
# Auxiliary function to convert numpy arrays to dictionaries
def convert_to_dictionary(a, t_start=0):
    temp_dictionary = {}

    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1+t_start)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1+t_start)] = a[dim0, dim1]

    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1+t_start)] = a[dim0]

    return temp_dictionary

In [ ]:
def get_timeseries(values, component: str):

    temp_data_idx = np.where(values == component)
    temp_data = values.iloc[temp_data_idx[0]].copy(deep=True).to_numpy()
    for i in np.arange(temp_data.shape[0]):
        temp_data[i, :] = pd.to_numeric(temp_data[i, :], errors='coerce')
    temp_data = temp_data[:, temp_data_idx[1][0] + 1:]

    return temp_data

In [ ]:
def get_characteristic(values, component: str, keep_string: bool = False):

    temp_data_idx = np.where(values == component)
    temp_data = values.iloc[temp_data_idx[0]].copy(deep=True).to_numpy()
    if not keep_string:
        for i in np.arange(temp_data.shape[0]):
            temp_data[i, :] = pd.to_numeric(temp_data[i, :], errors='coerce')
        temp_data = temp_data[:, temp_data_idx[1][0] + 1]
    else:
        for i in np.arange(temp_data.shape[0]):
            temp_data[i, :] = temp_data[i, :].astype(str)
        temp_data = temp_data[:, temp_data_idx[1][0] + 1]

    return temp_data

In [ ]:
model = pe.ConcreteModel()
model.t = pe.Set(initialize=np.arange(1, 25),
            doc='Time periods')

In [ ]:

conf = dotenv.dotenv_values()
executable = conf["PATH_TO_SOLVER"]

In [ ]:
simpler_model.params(model)
simpler_model.vars(model)
simpler_model.constraints(model)

In [ ]:
model.write(f'opt_logger.lp', io_options={'symbolic_solver_labels': True})
opt = pyomo.opt.SolverFactory('cplex', executable=executable, tee=True)
opt.options['LogFile'] = f'opt_logger.log'
opt.options['timelimit'] = 1*60
opt.solve(model)
log_infeasible_constraints(model)

In [ ]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())

    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)

    return s

In [ ]:
result_genActPower = ext_pyomo_vals(model.genActPower)
result_genActPower.to_csv('./result_genActPower.csv')

result_genExcActPower = ext_pyomo_vals(model.genExcPower)
result_genExcActPower.to_csv('./result_genExcActPower.csv')

result_pimp= ext_pyomo_vals(model.imports)
result_pimp.to_csv('./result_pimp.csv')

result_pexp = ext_pyomo_vals(model.exports)
result_pexp.to_csv('./result_pexp.csv')

result_P_import_relax = ext_pyomo_vals(model.P_import_relax)
result_P_import_relax.to_csv('./result_P_import_relax.csv')